# Analiza vremena glavnih evropskih mest
### Prijekt pri programiranju 1

In [202]:
import re

In [203]:
import pandas

In [204]:
import requests

In [205]:
import csv
import os
import sys

1.) Pripravi seznam velikih evropskih mest in ga uskladi z podatki iz spletne strani: 
http://www.weatherbase.com/weather/country.php3?r=EUR&regionname=Europe

In [ ]:
def popravi_zapis(niz):
    
    razdeli = niz.split()
    popravljeno = ('-').join(razdeli)
    
    if popravljeno == 'Vatican-City-(Holy-See)':
        popravljeno = 'Holy-See'
        
    elif popravljeno == 'Iceland':
        popravljeno = 'Faroe-Islands'
        
    elif popravljeno == 'Reykjavik':
        popravljeno = 'Torshavn'
        
    elif popravljeno == 'Tirana':
        popravljeno = 'Tirane'
        
    elif popravljeno == 'Kyiv':
        popravljeno = 'Kiev'
        
    return popravljeno

In [206]:

def glavna_mesta_drzav():
    
    '''
    Pripavi seznam drzave in enega izmed njenih najvecjih mest. (Ne nujno Prestolnice, kadar zanjo ni zadostnih podatkov).
    '''
    
    naslov = 'https://www.countries-ofthe-world.com/capitals-of-europe.html'
    r = requests.get(naslov)
    besedilo = r.text


    iskani_niz = r'<tr( class="grey"|)><td>(?P<drzava>\D*?)</td><td>(?P<prestolnica>\D*?)</td></tr>'
    iscemo = re.compile(iskani_niz , flags=re.DOTALL)
    
    slovarVelikihMest = dict()
    for najdeni in re.finditer(iscemo, besedilo):
        
        Prestolnica = popravi_zapis(najdeni.group('prestolnica'))
        Drzava = popravi_zapis(najdeni.group('drzava'))

        
        slovarVelikihMest[Drzava] = Prestolnica
    
    #Popravi izjeme:
    slovarVelikihMest['Monaco'] = 'Monte-Carlo'
                          
    return slovarVelikihMest
                          
Prestolnice = glavna_mesta_drzav()
#print(Prestolnice)

{'Faroe-Islands': 'Torshavn', 'Malta': 'Valletta', 'France': 'Paris', 'Ireland': 'Dublin', 'Portugal': 'Lisbon', 'Turkey': 'Ankara', 'Latvia': 'Riga', 'Serbia': 'Belgrade', 'Czech-Republic': 'Prague', 'Luxembourg': 'Luxembourg', 'Armenia': 'Yerevan', 'Belarus': 'Minsk', 'Germany': 'Berlin', 'United-Kingdom': 'London', 'Belgium': 'Brussels', 'Italy': 'Rome', 'Russia': 'Moscow', 'Albania': 'Tirane', 'Slovakia': 'Bratislava', 'Liechtenstein': 'Vaduz', 'Switzerland': 'Bern', 'Romania': 'Bucharest', 'Lithuania': 'Vilnius', 'Austria': 'Vienna', 'San-Marino': 'San-Marino', 'Slovenia': 'Ljubljana', 'Poland': 'Warsaw', 'Azerbaijan': 'Baku', 'Bosnia-and-Herzegovina': 'Sarajevo', 'Netherlands': 'Amsterdam', 'Denmark': 'Copenhagen', 'Estonia': 'Tallinn', 'Holy-See': 'Vatican-City', 'Norway': 'Oslo', 'Croatia': 'Zagreb', 'Andorra': 'Andorra-la-Vella', 'Finland': 'Helsinki', 'Georgia': 'Tbilisi', 'Kazakhstan': 'Astana', 'Montenegro': 'Podgorica', 'Hungary': 'Budapest', 'Macedonia': 'Skopje', 'Spain'

2.) KORAK: Priprava parametrov, ki so potrebni za dostop do vseh spletnih strani, ki jih želimo zajeti.

In [207]:
def najdi_drzave():
    
    '''
    Poisce parametre za URLje spletnih strani posameznih drzav.
    '''
    
    naslov = 'http://www.weatherbase.com/weather/country.php3?r=EUR&regionname=Europe'
    r = requests.get(naslov)
    besedilo = r.text
    
    slovar_parametrov = dict()
    
    iskani_niz = r'<li><a class="redglow" href="/weather/\D*?.php3\?c=(?P<kratica>\D*?)&name=(?P<ime>\D*?)" onClick="recordOutboundLink'
    iscemo = re.compile(iskani_niz , flags=re.DOTALL)
    
    for drzava in re.finditer(iscemo, besedilo):

        slovar_parametrov[drzava.group('ime')] = '{}&name={}'.format(drzava.group('kratica'),drzava.group('ime'))
        
    return slovar_parametrov

parametri = najdi_drzave()
#print(parametri)

{'Faroe-Islands': 'FO&name=Faroe-Islands', 'Malta': 'MT&name=Malta', 'France': 'FR&name=France', 'Ireland': 'IE&name=Ireland', 'Portugal': 'PT&name=Portugal', 'Turkey': 'TR&name=Turkey', 'Latvia': 'LV&name=Latvia', 'Serbia': 'RS&name=Serbia', 'Czech-Republic': 'CZ&name=Czech-Republic', 'Luxembourg': 'LU&name=Luxembourg', 'Germany': 'DE&name=Germany', 'United-Kingdom': 'GB&name=United-Kingdom', 'Belgium': 'BE&name=Belgium', 'Italy': 'IT&name=Italy', 'Russia': 'RU&name=Russia', 'Albania': 'AL&name=Albania', 'Slovakia': 'SK&name=Slovakia', 'Liechtenstein': 'LI&name=Liechtenstein', 'Iceland': 'IS&name=Iceland', 'Romania': 'RO&name=Romania', 'Lithuania': 'LT&name=Lithuania', 'Austria': 'AT&name=Austria', 'San-Marino': 'SM&name=San-Marino', 'Slovenia': 'SI&name=Slovenia', 'Poland': 'PL&name=Poland', 'Bosnia-and-Herzegovina': 'BA&name=Bosnia-and-Herzegovina', 'Netherlands': 'NL&name=Netherlands', 'Denmark': 'DK&name=Denmark', 'Estonia': 'EE&name=Estonia', 'Holy-See': 'VA&name=Holy-See', 'Norw

In [212]:
def najdi_mesto(besedilo, ime):
    
    '''
    Sprejme vsebino spletne strani in ime (drzava, mesto) ter poisce sifro mest.
    Zapiše jih v obliki parametrov za URLje podstrani.
    '''

    parameter = list()
    iskani_niz = r'<li><a class="redglow" href="/weather/weather.php3\?s=(?P<sifra>\d*?)&cityname=' + ime
    iscemo = re.compile(iskani_niz , flags=re.DOTALL)
    
    for najdeni in re.finditer(iscemo, besedilo):
        
        parameter += ['s={}&cityname={}'.format(najdeni.group('sifra'),ime)]
        
    return parameter

In [214]:
#NE DELA ZA UNITED KINGDOM!!!!!!!'c=GB&s=ENG&statename=England-United-Kingdom' 


def zajemi_drugo(parametri_drzav, slovarPrestolnic):
    
    '''
    Sprejme seznam parametrov za URLje drzav in iskanih mest,
    ki jih zajame in v njih pojisce koncne parametre.
    '''
    
    koncni_parametri = list()
    
    for drzava, parameter in parametri_drzav.items():
        
        for drzava2, prestolnica in slovarPrestolnic.items():
            

            if drzava == drzava2 and drzava != 'United-Kingdom':
                    
                ime = slovarPrestolnic[drzava] + '-' + drzava
                #print(ime)
        
                naslov = r'http://www.weatherbase.com/weather/city.php3?c=' + parameter
                r = requests.get(naslov)
                koncni_parametri += najdi_mesto(r.text, ime)
        
    return koncni_parametri

vsi_parametri = zajemi_drugo(parametri, Prestolnice)
#print(vsi_parametri)

['s=11060&cityname=Torshavn-Faroe-Islands', 's=79561&cityname=Valletta-Malta', 's=94170&cityname=Paris-France', 's=96930&cityname=Dublin-Ireland', 's=63580&cityname=Lisbon-Portugal', 's=82171&cityname=Ankara-Turkey', 's=22462&cityname=Riga-Latvia', 's=27231&cityname=Belgrade-Serbia', 's=81511&cityname=Prague-Czech-Republic', 's=9560&cityname=Luxembourg-Luxembourg', 's=58301&cityname=Berlin-Germany', 's=15460&cityname=Brussels-Belgium', 's=124261&cityname=Rome-Italy', 's=551572&cityname=Moscow-Russia', 's=51631&cityname=Tirane-Albania', 's=61811&cityname=Bratislava-Slovakia', 's=528523&cityname=Vaduz-Liechtenstein', 's=2451&cityname=Bucharest-Romania', 's=267300&cityname=Vilnius-Lithuania', 's=63011&cityname=Vienna-Austria', 's=528592&cityname=San-Marino-San-Marino', 's=41031&cityname=Ljubljana-Slovenia', 's=57321&cityname=Warsaw-Poland', 's=35331&cityname=Sarajevo-Bosnia-and-Herzegovina', 's=4260&cityname=Amsterdam-Netherlands', 's=8160&cityname=Copenhagen-Denmark', 's=83062&cityname=T

3.) KORAK: Funkcije, ki so v pomoč pri shranjevanju darotek in shranjevanje.

In [215]:
 def ustvari_mapo():

    '''
    Ustvari mapo z danim imenom.
    '''   
    
    if not os.path.exists('velikaMestaEvrope'):
        os.mkdir('velikaMestaEvrope')
        
ustvari_mapo()

In [216]:
def shranjevanje( parametri):

    '''
    Zajame vsebino spletnih strani in jo shrani v datoteke.
    '''
    
    imena_datotek = list()
    for parameter in parametri:
        
        
        #Izjema zaradi UnicodeEncodeError(\xf8):
        if parameter == 's=88410&cityname=Oslo-Norway':
            
            naslov = 'http://www.weatherbase.com/weather/weather.php3?' + parameter
            r = requests.get(naslov)
            
            besedilo = r.text.replace('\xf8','')
        
        #V splosnem:
        else:   

            naslov = 'http://www.weatherbase.com/weather/weather.php3?' + parameter
            r = requests.get(naslov)
            besedilo = r.text

        
        ime = parameter.split('=')
        pot = os.getcwd() + '\\' + 'velikaMestaEvrope'
        imena_datotek += [pot + '\\' + ime[2]]
        
        if (ime[0] + '.txt') not in os.listdir(pot):

            with open(pot + '\\' + ime[2] + '.txt','w') as f:
                f.write(besedilo)
                f.close()
                print('Datoteka ' + ime[2] + ' shranjena!')

    return imena_datotek


imena_dat = shranjevanje(vsi_parametri)


Datoteka Torshavn-Faroe-Islands shranjena!
Datoteka Valletta-Malta shranjena!
Datoteka Paris-France shranjena!
Datoteka Dublin-Ireland shranjena!
Datoteka Lisbon-Portugal shranjena!
Datoteka Ankara-Turkey shranjena!
Datoteka Riga-Latvia shranjena!
Datoteka Belgrade-Serbia shranjena!
Datoteka Prague-Czech-Republic shranjena!
Datoteka Luxembourg-Luxembourg shranjena!
Datoteka Berlin-Germany shranjena!
Datoteka Brussels-Belgium shranjena!
Datoteka Rome-Italy shranjena!
Datoteka Moscow-Russia shranjena!
Datoteka Tirane-Albania shranjena!
Datoteka Bratislava-Slovakia shranjena!
Datoteka Vaduz-Liechtenstein shranjena!
Datoteka Bucharest-Romania shranjena!
Datoteka Vilnius-Lithuania shranjena!
Datoteka Vienna-Austria shranjena!
Datoteka San-Marino-San-Marino shranjena!
Datoteka Ljubljana-Slovenia shranjena!
Datoteka Warsaw-Poland shranjena!
Datoteka Sarajevo-Bosnia-and-Herzegovina shranjena!
Datoteka Amsterdam-Netherlands shranjena!
Datoteka Copenhagen-Denmark shranjena!
Datoteka Tallinn-Esto

In [ ]:
#print(imena_dat)

In [217]:
def najdi_podatke(besedilo):

    '''
    Sprejme vsebino spletne strani in na njej poisce parametre,
    ki jih zelimo analizirati: id, drzavo, mesto, temperature (v fahrenheitih).
    '''

    iscemo = re.compile(r'<meta name="city" content="(?P<mesto>\D*?)">.*?'
                        r'<meta name="country" content="(?P<drzava>\D*?)">.*?'
                        r'http://www.weatherbase.com/weather/weather.php3\?s=(?P<id>\d+)&cityname=.*?'
                        r'type=Average.Temperature&units=Fahrenheit.*?symbol=F&data=(?P<temperature>.*?)"><img'
    , flags=re.DOTALL)

    for najdeni in re.finditer(iscemo, besedilo):
        
        return [[najdeni.group('mesto'),najdeni.group('drzava'),najdeni.group('id'),najdeni.group('temperature')]]


In [219]:
def poisciPodatke(imena_datotek):

    '''
    Sprejme imena datotek v katerih poisce iskane podatke in jih zapise v seznam.
    '''

    podatki = list()

    for ime in imena_datotek:

        with open(ime + '.txt', 'r') as f:
            podatki += najdi_podatke(f.read())

    return podatki

podatki1 = poisciPodatke(imena_dat)
#print(podatki1)

[['Torshavn', 'Faroe Islands', '11060', '38,39,39,41,45,48,51,52,48,45,41,39'], ['Valletta', 'Malta', '79561', '54,54,56,60,66,73,79,79,76,70,62,57'], ['Paris', 'France', '94170', '41,42.1,47.8,52.7,59.5,64.9,68.9,68.7,62.4,55.4,46.9,41.9'], ['Dublin', 'Ireland', '96930', '41.5,41.5,44.2,46.9,51.6,56.5,60.1,59.5,56.1,50.9,45.3,42.1'], ['Lisbon', 'Portugal', '63580', '52.3,54.7,57.7,59.5,63.1,68.5,72.9,73.4,71.1,65.1,58.6,54.3'], ['Ankara', 'Turkey', '82171', '27,32,40,49,56,63,69,69,62,52,41,33'], ['Riga', 'Latvia', '22462', '26,26,33,42,53,60,63,62,54,45,37,29'], ['Belgrade', 'Serbia', '27231', '34.5,37.6,45.7,55.2,64.6,69.8,73.4,72.9,64.4,55.2,44.8,36.9'], ['Prague', 'Czech Republic', '81511', '29,30,39,45,55,60,63,63,57,47,37,32'], ['Luxembourg', 'Luxembourg', '9560', '33,34,41,45,54,59,63,63,57,49,39,35'], ['Berlin', 'Germany', '58301', '31,33,40,46,55,62,65,64,58,49,40,34'], ['Brussels', 'Belgium', '15460', '38,38,44,48,55,60,64,64,59,52,44,40'], ['Rome', 'Italy', '124261', '46.9,

4.) KORAK: Podatke pretvori v ustrezno obliko in jih zapiše v CSV datoteko.

In [220]:
def iz_fahrenheit_to_celzij(niz):

    '''
    Sprejme niz vrednostih v fahrenheitih, ločenih z vejicami in vrne seznam vrednosti v celzijih
    '''

    seznam_ne_pretvorjenih = niz.split(',')
    ze_pretvorjeni = list()
    celzij = 0

    for fahrenheit in seznam_ne_pretvorjenih:

        celzij = round((float(fahrenheit) - 32) * (5 / 9),1)
        ze_pretvorjeni += [str(celzij)]

    return ze_pretvorjeni

In [221]:
def povprecje(sez):

    '''
    Sprejme seznam nizov stevil in vrne njihovo povprecno vrednost.
    '''

    povprecna_vrednost = 0

    for stevilo in sez:
        povprecna_vrednost += float(stevilo)

    return str(povprecna_vrednost // len(sez))

In [222]:
def uredi_podatke(podatki):

    '''
    V vhodnih podatkih popravi temperature in doda njihovo letno povprecje.
    '''

    urejeniPodatki = list()

    for mesto in podatki:

        temperature = iz_fahrenheit_to_celzij(mesto[3])
        povprecnaLetna = povprecje(temperature)
        urejeniPodatki += [mesto[:3] + temperature + [povprecnaLetna]]

    return urejeniPodatki

podatki2 = uredi_podatke(podatki1)
#print(podatki2)

[['Torshavn', 'Faroe Islands', '11060', '3.3', '3.9', '3.9', '5.0', '7.2', '8.9', '10.6', '11.1', '8.9', '7.2', '5.0', '3.9', '6.0'], ['Valletta', 'Malta', '79561', '12.2', '12.2', '13.3', '15.6', '18.9', '22.8', '26.1', '26.1', '24.4', '21.1', '16.7', '13.9', '18.0'], ['Paris', 'France', '94170', '5.0', '5.6', '8.8', '11.5', '15.3', '18.3', '20.5', '20.4', '16.9', '13.0', '8.3', '5.5', '12.0'], ['Dublin', 'Ireland', '96930', '5.3', '5.3', '6.8', '8.3', '10.9', '13.6', '15.6', '15.3', '13.4', '10.5', '7.4', '5.6', '9.0'], ['Lisbon', 'Portugal', '63580', '11.3', '12.6', '14.3', '15.3', '17.3', '20.3', '22.7', '23.0', '21.7', '18.4', '14.8', '12.4', '17.0'], ['Ankara', 'Turkey', '82171', '-2.8', '0.0', '4.4', '9.4', '13.3', '17.2', '20.6', '20.6', '16.7', '11.1', '5.0', '0.6', '9.0'], ['Riga', 'Latvia', '22462', '-3.3', '-3.3', '0.6', '5.6', '11.7', '15.6', '17.2', '16.7', '12.2', '7.2', '2.8', '-1.7', '6.0'], ['Belgrade', 'Serbia', '27231', '1.4', '3.1', '7.6', '12.9', '18.1', '21.0', '

In [223]:
def pretvori_v_csv(podatki):

    '''
    Podatke iz seznama seznamov zapise v csv datoteko.
    '''

    with open('podatki.txt','w') as csvFile:
        naslovnaVrstica = 'MESTO,Drzava,sifra,januar,februar,marec,april,maj,juni,juli,avgust,' \
        'september,oktober,november,december, LETNO POVPREČJE' + '\n'
        csvFile.write(naslovnaVrstica)
        for vrstica in podatki:
            csvFile.write(','.join(vrstica) + '\n')

    csvFile.close()

pretvori_v_csv(podatki2)

5.) Analiza podatkov s pandasi

In [224]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [225]:
temperature_df = pd.read_csv('podatki.txt', encoding='latin1')

In [226]:
temperature_df[:]

,MESTO,Drzava,sifra,januar,februar,marec,april,maj,juni,juli,avgust,september,oktober,november,december,LETNO POVPREÈJE
0,Torshavn,Faroe Islands,11060,3.3,3.9,3.9,5.0,7.2,8.9,10.6,11.1,8.9,7.2,5.0,3.9,6.0
1,Valletta,Malta,79561,12.2,12.2,13.3,15.6,18.9,22.8,26.1,26.1,24.4,21.1,16.7,13.9,18.0
2,Paris,France,94170,5.0,5.6,8.8,11.5,15.3,18.3,20.5,20.4,16.9,13.0,8.3,5.5,12.0
3,Dublin,Ireland,96930,5.3,5.3,6.8,8.3,10.9,13.6,15.6,15.3,13.4,10.5,7.4,5.6,9.0
4,Lisbon,Portugal,63580,11.3,12.6,14.3,15.3,17.3,20.3,22.7,23.0,21.7,18.4,14.8,12.4,17.0
5,Ankara,Turkey,82171,-2.8,0.0,4.4,9.4,13.3,17.2,20.6,20.6,16.7,11.1,5.0,0.6,9.0
6,Riga,Latvia,22462,-3.3,-3.3,0.6,5.6,11.7,15.6,17.2,16.7,12.2,7.2,2.8,-1.7,6.0
7,Belgrade,Serbia,27231,1.4,3.1,7.6,12.9,18.1,21.0,23.0,22.7,18.0,12.9,7.1,2.7,12.0
8,Prague,Czech Republic,81511,-1.7,-1.1,3.9,7.2,12.8,15.6,17.2,17.2,13.9,8.3,2.8,0.0,8.0
9,Luxembourg,Luxembourg,9560,0.6,1.1,5.0,7.2,12.2,15.0,17.2,17.2,13.9,9.4,3.9,1.7,8.0
